# Mounting movement
This notebook was used to create the lunging movement. It's content was later moved to ``final_courtship_scenario.ipynb``.

In [1]:
import numpy as np
from tqdm import trange
from gymnasium import spaces
from gymnasium.utils.env_checker import check_env

from flygym.simulation import SingleFlySimulation, Fly
from flygym.examples.common import PreprogrammedSteps
from flygym.examples.cpg_controller import CPGNetwork
from flygym.preprogrammed import get_cpg_biases
from hybrid_turning_fly import HybridTurningFly
from pathlib import Path
import matplotlib.pyplot as plt
from flygym import Parameters, Camera, SingleFlySimulation, Fly
from flygym.examples.turning_controller import HybridTurningNMF
from flygym.simulation import Simulation
from hybrid_turning_fly import HybridTurningFly
from movodor_arena import MovOdorArena



sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.5,
    render_window_size=(800, 608),
    enable_olfaction=True,
    enable_adhesion=True,
    draw_adhesion=False,
    render_camera="Animat/camera_left",
)
timestep = sim_params.timestep
fly_male = Fly(
    name="male",
    
    enable_adhesion=True,
    enable_olfaction=True,
    spawn_pos=(0, 0, 0.2),
)

cam = Camera(
    fly=fly_male,
    camera_id="Animat/camera_left",
    window_size=(800, 608),
    play_speed=0.5,
)
seed=1
# Odor source: array of shape (num_odor_sources, 3) - xyz coords of odor sources
odor_source = np.array([[24, 10, 1.5]])

# Peak intensities: array of shape (num_odor_sources, odor_dimensions)
# For each odor source, if the intensity is (x, 0) then the odor is in the 1st dimension
# (in this case attractive). If it's (0, x) then it's in the 2nd dimension (in this case
# aversive)
peak_odor_intensity = np.array([[1, 0]])

# Marker colors: array of shape (num_odor_sources, 4) - RGBA values for each marker,
# normalized to [0, 1]
marker_colors = [[255, 127, 14]]
marker_colors = np.array([[*np.array(color) / 255, 1] for color in marker_colors])

odor_dimensions = len(peak_odor_intensity[0])
arena=arena = MovOdorArena(
    size=(300, 300),
    friction=(1, 0.005, 0.0001),
    num_sensors=4,
    move_speed=0,
    move_direction="right",
    odor_source=odor_source,
    peak_intensity=peak_odor_intensity,
    diffuse_func=lambda x: x**-2,
    marker_colors=marker_colors,
    marker_size=0.3,
)

sim = SingleFlySimulation(
    fly=fly_male,
    cameras=[cam],
    arena=arena,
    timestep=timestep,
)

preprogrammed_steps=PreprogrammedSteps()
swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

standing_action = []
for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(
            preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1])
        )
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

stand_action = {"joints": standing_action, "adhesion": np.zeros(len(legs))}

# Let the fly stand on the floor first
for i in range(int(0.2 // sim.timestep)):
    sim.step(stand_action)


run_time = 0.5
target_num_steps = int(run_time // sim.timestep)

foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], 2 * np.pi, target_num_steps)
hind_swing_ids = np.linspace(0.0, swing_periods["RH"][1], target_num_steps)

R_midleg_start = preprogrammed_steps.get_joint_angles("RM", swing_periods["RM"][1])
R_midleg_stretch = np.linspace(
    np.zeros(len(R_midleg_start)), -R_midleg_start, target_num_steps
)

L_midleg_start = preprogrammed_steps.get_joint_angles("LM", swing_periods["LM"][1])
L_midleg_stretch = np.linspace(
    np.zeros(len(L_midleg_start)), -L_midleg_start, target_num_steps
)

adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])

all_joint_angles = []

for i in range(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("F"):
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i])
            )
        elif leg.endswith("M"):
            midleg_joint_angles = preprogrammed_steps.get_joint_angles(
                leg, middle_stance_ids[i]
            )
            if leg.startswith("R"):
                midleg_joint_angles += R_midleg_stretch[i]
            elif leg.startswith("L"):
                midleg_joint_angles += L_midleg_stretch[i]

            joint_angles.extend(midleg_joint_angles)
        else:
            joint_angles.extend(
                preprogrammed_steps.get_joint_angles(leg, hind_swing_ids[i])
            )

    all_joint_angles.append(joint_angles.copy())

    action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
    sim.step(action)
    sim.render()

from IPython.display import Video

cam.save_video("./video/mounting/mounting_video.mp4")

Video("./video/mounting/mounting_video.mp4", width=800, height=608)